In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import json
from tqdm import tqdm

_transport = RequestsHTTPTransport(
    url=url1,
    use_json=True,
)

client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)

## DATA

In [2]:
# Define the query
query = gql('''
query MyQuery {
  eraValidators {
    validatorId
  }
}
''')

# Execute the query
result = client.execute(query)

# Convert the result into a pandas DataFrame
df = pd.json_normalize(result['eraValidators'])

# Write the result to a file
with open('result.json', 'w') as f:
    json.dump(result, f)

# Print the DataFrame
print(df.nunique())

validatorId    1186
dtype: int64


In [8]:
query = gql('''
query MyQuery {
  eraValidators {
    validatorId
    totalBonded
    selfBonded
    nominatorsCount
  }
}
''')


result = client.execute(query)

In [ ]:
result

In [4]:
# query = gql('''
# {
#   __schema {
#     types {
#       name
#       fields {
#         name
#         type {
#           name
#           kind
#           ofType {
#             name
#             kind
#           }
#         }
#       }
#     }
#   }
# }
# ''')
# client.execute(query)

In [ ]:
import pandas as pd

# Create an empty DataFrame
df = pd.DataFrame()

# Iterate over eras from 1 to 1125
for era_id in range(1, 5):
    # Construct the query for the current era
    query = '''
    query MyQuery {
      eraValidators(where: {era: {id_eq: "%d"}}) {
        era {
          validators {
            validatorId
            totalBonded
            selfBonded
            nominatorsCount
          }
          id
        }
      }
    }
    ''' % era_id

    # Execute the query and get the result
    result = client.execute(gql(query))

    # Extract the relevant data from the result
    era_validators = result['data']['eraValidators'][0]['era']['validators']

    # Convert the data to a DataFrame
    era_df = pd.DataFrame(era_validators)

    # Add the era id as a column
    era_df['era_id'] = era_id

    # Concatenate the current era DataFrame with the main DataFrame
    df = pd.concat([df, era_df], ignore_index=True)


In [ ]:
import pandas as pd

# Create an empty DataFrame
df = pd.DataFrame()

# Iterate over eras from 1 to 1125
for era_data in result['eraValidators']:
    # Extract the era id
    era_id = era_data['era']['id']

    # Extract the validators for the current era
    era_validators = era_data['era']['validators']

    # Convert the data to a DataFrame
    era_df = pd.DataFrame(era_validators)

    # Add the era id as a column
    era_df['era_id'] = era_id

    # Concatenate the current era DataFrame with the main DataFrame
    df = pd.concat([df, era_df], ignore_index=True)

# Print the resulting DataFrame
print(df)


In [ ]:
# Create an empty DataFrame
df = pd.DataFrame()

# Iterate over eras from 1 to 1125
for era_id in tqdm(range(232, 1125)):
    # Construct the query for the current era
    query = '''
    query MyQuery {
      eraValidators(where: {era: {id_eq: "%d"}}) {
        era {
          validators {
            validatorId
            totalBonded
            selfBonded
            nominatorsCount
          }
          id
        }
      }
    }
    ''' % era_id

    # Execute the query and get the result
    result = client.execute(gql(query))

    # Check if the result contains the 'eraValidators' key
    if 'eraValidators' in result:
        era_data = result['eraValidators']

        # Iterate over each era in the result
        for era in era_data:
            era_id = era['era']['id']
            validators = era['era']['validators']

            # Convert the validators data to a DataFrame
            era_df = pd.DataFrame(validators)

            # Add the era id as a column
            era_df['era_id'] = era_id

            # Concatenate the current era DataFrame with the main DataFrame
            df = pd.concat([df, era_df], ignore_index=True)

In [24]:
df.tail()

,validatorId,totalBonded,selfBonded,nominatorsCount,era_id
10371797,12DsYUto9AcKA4kRz1yLcGh13CTLe7LbUjDkMS8ZY8rCK4rn,16369417383909862,100000000000,7,232
10371798,15Q4hMef4AGNKwgK5hd7k6zWvyaK8DhfhpaCHDGxdS7buxDf,21348038645804778,100000039437,31,232
10371799,14MQcaQQSptK9iriyMYFWLQezFjFihyrTg4YoKGyeNm7P34M,16474410389583593,10000000000,7,232
10371800,13mk2P1S54TiyKFrbMZwGW799zMagjPNW98N1REfxr4T6EeT,16476194462735423,10000000000,2,232
10371801,14yx4vPAACZRhoDQm1dyvXD3QdRQyCRRCe5tj1zPomhhS29a,21350317876204412,10000000000,16,232
